In [1]:
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def read_csv(day_trade):
    data_up = []
    data_down = []
    path = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/Train_Test_Data/ML_data_2014'
    for j,i in enumerate(day_trade):
        for k in range(0,len(i),1):
            path_up = path + '_' + str(j+1) + '_' + str(i[k]) + '_' + 'UP' + '.csv'
            path_down = path + '_' + str(j+1) + '_' + str(i[k]) + '_' + 'DOWN' + '.csv'
            data_up.append(pd.read_csv(path_up))
            data_down.append(pd.read_csv(path_down))
            #print path_down
    return data_up,data_down

In [3]:
day_trade = [[2,3,6,7,8,10,13,14,15,16,17,20,21,22,23,24,27,28,29,30],\
             [7,10,11,12,13,17,18,19,21,24,25,26,27],\
             [3,4,5,6,7,10,11,13,14,17,18,19,20,24,25,26,27,31]]

In [4]:
data_2014_up, data_2014_down = read_csv(day_trade)

In [6]:
from sklearn.grid_search import (GridSearchCV, RandomizedSearchCV)
from sklearn import (metrics, cross_validation, linear_model, preprocessing)
from sklearn import svm, datasets

In [7]:
i = 0
day = 0
latest_min = 60 * 30
pred_sec = 5

data_train = data_2014_up[day][i:i+latest_min]
X_train = data_train.drop(["0"],axis=1)
y_train = data_train['0']
data_test = data_2014_up[day][i+latest_min:i+latest_min+pred_sec]
X_test = data_test.drop(["0"],axis=1)
y_test = data_test['0']

In [51]:
turned_parameters = [{'kernel':['rbf'],'gamma':[1e-1,1e-2],'C':[1,10]},
                     {'kernel':['linear'],'C':[1,10]}]

In [45]:
svr = svm.SVC(probability=True)

In [47]:
Grid.estimator.get_params

<bound method SVC.get_params of SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)>

In [48]:
%time Grid.fit(X_train,y_train)

CPU times: user 27.3 s, sys: 0 ns, total: 27.3 s
Wall time: 28.1 s


GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'kernel': ['rbf'], 'C': [1, 10, 100], 'gamma': [0.1, 0.01, 0.001]}, {'kernel': ['linear'], 'C': [1, 10, 100]}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [49]:
Grid.best_params_,Grid.best_score_,Grid.grid_scores_

({'C': 1, 'gamma': 0.1, 'kernel': 'rbf'},
 0.8194444444444444,
 [mean: 0.81944, std: 0.12131, params: {'kernel': 'rbf', 'C': 1, 'gamma': 0.1},
  mean: 0.79333, std: 0.12063, params: {'kernel': 'rbf', 'C': 1, 'gamma': 0.01},
  mean: 0.72500, std: 0.08540, params: {'kernel': 'rbf', 'C': 1, 'gamma': 0.001},
  mean: 0.81500, std: 0.11079, params: {'kernel': 'rbf', 'C': 10, 'gamma': 0.1},
  mean: 0.81167, std: 0.13923, params: {'kernel': 'rbf', 'C': 10, 'gamma': 0.01},
  mean: 0.78167, std: 0.13956, params: {'kernel': 'rbf', 'C': 10, 'gamma': 0.001},
  mean: 0.80389, std: 0.11242, params: {'kernel': 'rbf', 'C': 100, 'gamma': 0.1},
  mean: 0.78778, std: 0.12189, params: {'kernel': 'rbf', 'C': 100, 'gamma': 0.01},
  mean: 0.79667, std: 0.16202, params: {'kernel': 'rbf', 'C': 100, 'gamma': 0.001},
  mean: 0.80333, std: 0.10952, params: {'kernel': 'linear', 'C': 1},
  mean: 0.81111, std: 0.09755, params: {'kernel': 'linear', 'C': 10},
  mean: 0.79111, std: 0.10890, params: {'kernel': 'linear', 

In [50]:
metrics.accuracy_score(Grid.predict(X_test),y_test)

1.0

In [106]:
latest_sec = 60 * 35
pred_sec = 15

Precision_Mean_Day = []
Precision_Std_Day = []
Feature_Importance = []
Feature_Importance_Day_one = []
Feature_Importance_Day_two = []
Feature_Importance_Day_three = []
Feature_Importance_Day_four = []
Feature_Importance_Day_five = []
Predict_Values_Day = []
True_Values_Day = []
for day in range(0,1,1):
    Precision = []
    Feature_Importance_one = []
    Feature_Importance_two = []
    Feature_Importance_three = []
    Feature_Importance_four = []
    Feature_Importance_five = []
    predict_values = []
    true_values = []
    for i in range(0,9000-latest_sec-pred_sec,pred_sec):

        print '----------------------------Rolling Window Time = %s----------------------------'%(i/pred_sec)
        data_train = data_2014_up[day][i:i+latest_min]
        X_train = data_train.drop(["0"],axis=1)
        y_train = data_train['0']

        data_test = data_2014_up[day][i+latest_sec:i+latest_sec+pred_sec]
        X_test = data_test.drop(["0"],axis=1)
        y_test = data_test['0']

        # hyper-parameter
        model_grid = [#{'kernel':['rbf'],'gamma':[1e-1,1e-2],'C':[1,10]},
                      {'kernel':['linear'],'C':[1,10]}]
        model = svm.SVC(probability=True)
        Grid = GridSearchCV(model,model_grid,cv=5,scoring = 'precision')
        %time Grid.fit(X_train,y_train) 
        print Grid.best_params_
        print 'CV Best Score = %s'%(Grid.best_score_)

        # training & testing
        model.set_params(**Grid.best_params_)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        print 'Prediction latest 15 second = %s'%(predictions)
        predict_values.append(predictions.tolist())
        true_values.append(y_test.tolist())
        prec = metrics.precision_score(y_test, predictions)
        print 'Precision = %s'%(prec)
        Precision.append(prec)
        if Grid.best_params_.values()[0] == 'linear':
            feature_imp = dict(zip([i for i in range(0,64,1)],model.coef_[0]))
            Top_five = sorted(feature_imp.items(),key = lambda x : x[1] , reverse=True)[0:5]
            print 'Kernel is linear and top five importance features = %s'%(Top_five)
            Feature_Importance_one.append(Top_five[0][0])
            Feature_Importance_two.append(Top_five[1][0])
            Feature_Importance_three.append(Top_five[2][0])
            Feature_Importance_four.append(Top_five[3][0])
            Feature_Importance_five.append(Top_five[4][0])
        else:
            print 'Kernel is rbf'

    print 'Day = %s, mean = %s, std = %s'%(day,np.mean(Precision),np.std(Precision))
    Precision_Mean_Day.append(Accuracy)
    Precision_Std_Day.append(Accuracy)
    Feature_Importance_Day_one.append(Feature_Importance_one)
    Feature_Importance_Day_two.append(Feature_Importance_two)
    Feature_Importance_Day_three.append(Feature_Importance_three)
    Feature_Importance_Day_four.append(Feature_Importance_four)
    Feature_Importance_Day_five.append(Feature_Importance_five)
    Predict_Values_Day.append(predict_values)
    True_Values_Day.append(true_values)

----------------------------Rolling Window Time = 0----------------------------
CPU times: user 8.94 s, sys: 38.7 ms, total: 8.98 s
Wall time: 11.2 s
{'kernel': 'linear', 'C': 10}
CV Best Score = 0.863196745489
Prediction latest 15 second = [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
Precision = 1.0
Kernel is linear and top five importance features = [(28, 9.1075574173486142), (24, 7.1406342083809067), (8, 5.5218908607106814), (25, 5.4318121293271275), (63, 4.8616358288778025)]
----------------------------Rolling Window Time = 1----------------------------
CPU times: user 7.53 s, sys: 55 ms, total: 7.58 s
Wall time: 9.88 s
{'kernel': 'linear', 'C': 10}
CV Best Score = 0.858673079443
Prediction latest 15 second = [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
Precision = 1.0
Kernel is linear and top five importance features = [(28, 8.8744125719974196), (24, 7.1142396413621523), (8, 5.593945304661978), (25, 5.4130014616933275), (63, 5.2955458338543497)]
-

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


CPU times: user 16.6 s, sys: 56.9 ms, total: 16.7 s
Wall time: 46 s
{'kernel': 'linear', 'C': 1}
CV Best Score = 0.900333872097
Prediction latest 15 second = [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Precision = 0.0
Kernel is linear and top five importance features = [(28, 4.4648815386429606), (18, 4.2227731607911263), (27, 3.6498677743901609), (1, 3.5252080199008655), (0, 3.4523846924710635)]
----------------------------Rolling Window Time = 48----------------------------


KeyboardInterrupt: 

In [58]:
Grid.best_params_.values()[0] == 'linear'

False

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [107]:
predict_values

[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.

In [108]:
true_values

[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [ ]:
latest_sec = 60 * 35
pred_sec = 15

Accuracy_Mean_Day = []
Accuracy_Std_Day = []
Feature_Importance = []
Feature_Importance_Day_one = []
Feature_Importance_Day_two = []
Feature_Importance_Day_three = []
Feature_Importance_Day_four = []
Feature_Importance_Day_five = []
Predict_Values_Day = []
True_Values_Day = []

for day in range(0,1,1):
    Accuracy = []
    Feature_Importance_one = []
    Feature_Importance_two = []
    Feature_Importance_three = []
    Feature_Importance_four = []
    Feature_Importance_five = []
    predict_values = []
    true_values = []
    for i in range(0,9000-latest_sec-pred_sec,pred_sec):

        #print '----------------------------Rolling Window Time = %s----------------------------'%(i/pred_sec)
        data_train = data_2014_up[day][i:i+latest_min]
        X_train = data_train.drop(["0"],axis=1)
        y_train = data_train['0']

        data_test = data_2014_up[day][i+latest_sec:i+latest_sec+pred_sec]
        X_test = data_test.drop(["0"],axis=1)
        y_test = data_test['0']

        # hyper-parameter
        model_grid = [#{'kernel':['rbf'],'gamma':[1e-1],'C':[1,10]},
                      {'kernel':['linear'],'C':[10]}]
        model = svm.SVC(probability=True)
        Grid = GridSearchCV(model,model_grid,cv=5,scoring = 'accuracy')
        #%time 
        Grid.fit(X_train,y_train) 
        #print Grid.best_params_
        #print 'CV Best Score = %s'%(Grid.best_score_)

        # training & testing
        model.set_params(**Grid.best_params_)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        #print 'Prediction latest 15 second = %s'%(predictions)
        predict_values.append(predictions.tolist())
        true_values.append(y_test.tolist())
        acc = metrics.accuracy_score(y_test, predictions)
        #print 'Accuracy = %s'%(acc)
        Accuracy.append(acc)
        if Grid.best_params_.values()[0] == 'linear':
            feature_imp = dict(zip([i for i in range(0,64,1)],model.coef_[0]))
            Top_five = sorted(feature_imp.items(),key = lambda x : x[1] , reverse=True)[0:5]
            #print 'Kernel is linear and top five importance features = %s'%(Top_five)
            Feature_Importance_one.append(Top_five[0][0])
            Feature_Importance_two.append(Top_five[1][0])
            Feature_Importance_three.append(Top_five[2][0])
            Feature_Importance_four.append(Top_five[3][0])
            Feature_Importance_five.append(Top_five[4][0])
        else:
            print 'Kernel is rbf'

    print 'Day = %s, mean = %s, std = %s'%(day,np.mean(Accuracy),np.std(Accuracy))
    Accuracy_Mean_Day.append(Accuracy)
    Accuracy_Std_Day.append(Accuracy)
    Feature_Importance_Day_one.append(Feature_Importance_one)
    Feature_Importance_Day_two.append(Feature_Importance_two)
    Feature_Importance_Day_three.append(Feature_Importance_three)
    Feature_Importance_Day_four.append(Feature_Importance_four)
    Feature_Importance_Day_five.append(Feature_Importance_five)
    Predict_Values_Day.append(predict_values)
    True_Values_Day.append(true_values)